In [3]:
import cv2
import numpy as np

In [4]:
from keras.models import load_model

In [5]:
model=load_model('MRL.h5')

In [6]:
import time

In [11]:
from tkinter import *

def mk(cap,face_cascade,l,font,model):
    root = Tk()
    mylabel=Label(root,text="Please close your eyes!")
    mylabel.config(font=("Courier", 44))
    mylabel.pack()
    def task():
        while(cap.isOpened()):
            ret, frame = cap.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            for (x, y, w, h) in faces:
                im=gray[y+h//4: y+h//2,x+w//8:x+w//2]
                im=Image.fromarray(im)
                im=im.resize((80,80))
                im=np.array(im)
                im=im.reshape(1,80,80,1)
                p = model.predict_classes(im)
                overlay_text = "%s " % (l[p[0]])
                cv2.rectangle(frame, (x+w//8, y+h//4), (x+w//2, y+h//2), (255, 0, 0), 2)
                cv2.putText(frame, overlay_text, (x+w//6, y+h//4), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.imshow('frame',frame)
            if(p[0]==0):
                break
        root.destroy()
    root.after(100, task)
    root.mainloop()

In [12]:
from PIL import Image

In [15]:
n=10
cap = cv2.VideoCapture(700)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
l=['closed','open']
start=time.time() 
font = cv2.FONT_HERSHEY_SIMPLEX
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        im=gray[y+h//4: y+h//2,x+w//8:x+w//2]
        im=Image.fromarray(im)
        im=im.resize((80,80))
        im=np.array(im)
        im=im.reshape(1,80,80,1)
        p = model.predict_classes(im)
        overlay_text = "%s " % (l[p[0]])
        cv2.rectangle(frame, (x+w//8, y+h//4), (x+w//2, y+h//2), (255, 0, 0), 2)
        cv2.putText(frame, overlay_text, (x+w//6, y+h//4), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('frame',frame)
        if(p[0]==0):
            start=time.time()
        elif((time.time()-start)>=n):
            mk(cap,face_cascade,l,font,model) 
            start=time.time()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()